In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

#### Read

In [3]:
base_path = 'Target/'
dim_city = pd.read_csv(base_path + 'DimCity.csv')
dim_customer = pd.read_csv(base_path + 'DimCustomer.csv')
dim_date = pd.read_csv(base_path + 'DimDate.csv')
dim_product_catalog = pd.read_csv(base_path + 'DimProduct.csv')
dim_return_reason = pd.read_csv(base_path + 'DimReturnReason.csv')
dim_sales_agent = pd.read_csv(base_path + 'DimSalesAgent.csv')
dim_state = pd.read_csv(base_path + 'DimState.csv')
dim_order_status = pd.read_csv(base_path + 'DimOrderStatus.csv')
fact_order = pd.read_csv(base_path + 'FactOrder.csv')
dim_order_status

,OrderStatusKey,OrderStatusName
0,1,Delivered
1,2,Shipped
2,3,Cancelled
3,4,In Transit


#### Define Functions

In [4]:
class ColumnNames(enumerate):
    StateKey = 'StateKey'
    StateName = 'State'
    ReturnReasonKey = 'ReturnReasonKey'
    ReturnReasonName = 'ReturnReasonName'
    SaleAgentName = 'SaleAgentName'
    SaleAgentKey = 'SaleAgentKey'
    ProductCatalogKey = 'ProductCatalogKey'
    ProductCatalogName = 'ProductCatalogName'
    DateKey = 'DateKey'
    Date = 'Date'
    Day = 'Day'
    Month = 'Month'
    WeekDayName = 'WeekDayName'
    CityKey = 'CityKey'
    CityName = 'CityName'
    CustomerKey = 'CustomerKey'
    CustomerFName = 'Customer First Name'
    CustomerLName = 'Customer Last Name'
    CustomerPhone = 'Phone'
    OrderStatusKey = 'OrderStatusKey'
    OrderStatusName = 'OrderStatusName'
    OrderID = 'OrderID'
    OrderDate = 'OrderDate'
    QuantityOrdered = 'Quantity Ordered'
    ManufacturerPrice = 'Manufacturer Price'
    SalePrice = 'Sale Price'
    ProfitPerUnit = 'Total Profit (GMROI)'
    ProfitPerOrder = 'Profit Per Order'

In [5]:
def get_fact_view(fact_df: pd.DataFrame, key_columns_names: list[ColumnNames]):
    fact_view = fact_df.copy()
    if ColumnNames.DateKey in key_columns_names:
        fact_view = fact_view.join(dim_date.set_index(ColumnNames.DateKey), on=ColumnNames.DateKey)
    if ColumnNames.CustomerKey in key_columns_names:
        fact_view = fact_view.join(dim_customer.set_index(ColumnNames.CustomerKey), on=ColumnNames.CustomerKey)
    if ColumnNames.CityKey in key_columns_names:
        fact_view = fact_view.join(dim_city[[ColumnNames.CityKey, ColumnNames.CityName]].set_index(ColumnNames.CityKey), on=ColumnNames.CityKey)
    if ColumnNames.StateKey in key_columns_names:
        fact_view = fact_view.join(dim_state.set_index(ColumnNames.StateKey), on=ColumnNames.StateKey)
    if ColumnNames.SaleAgentKey in key_columns_names:
        fact_view = fact_view.join(dim_sales_agent.set_index(ColumnNames.SaleAgentKey), on=ColumnNames.SaleAgentKey)
    if ColumnNames.ProductCatalogKey in key_columns_names:
        fact_view = fact_view.join(dim_product_catalog.set_index(ColumnNames.ProductCatalogKey), on=ColumnNames.ProductCatalogKey)
    if ColumnNames.ReturnReasonKey in key_columns_names:
        fact_view = fact_view.join(dim_return_reason.set_index(ColumnNames.ReturnReasonKey), on=ColumnNames.ReturnReasonKey)
    if ColumnNames.OrderStatusKey in key_columns_names:
        fact_view = fact_view.join(dim_order_status.set_index(ColumnNames.OrderStatusKey), on=ColumnNames.OrderStatusKey)
    #remove key columns 
    for col in key_columns_names:
        fact_view.drop(col, axis=1, inplace=True)
    
    return fact_view

In [6]:
import statsmodels.api as sm 
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
def anova_pairwise_tukeyhsd(df, col1: str, col2: str, no_plot=False):
    # colname cannot have white space
    df_temp = df.copy()
    col1_replaced = col1.replace(' ', '').replace('(', '').replace(')', '')
    col2_replaced = col2.replace(' ', '').replace('(', '').replace(')', '')
    df_temp.rename(columns={col1: col1_replaced, col2: col2_replaced}, inplace=True)
    print(f'ANOVA for {col1_replaced} and {col2_replaced}')
    if not no_plot:
        fig = plt.figure(figsize=(4, 2))
        ax1 = fig.add_subplot(1, 1, 1)
        sns.boxplot(x=col1_replaced, y=col2_replaced, data=df_temp, palette="Set3", ax=ax1)
        plt.show()

    model = ols(f'{col2_replaced} ~ C({col1_replaced})', data=df_temp).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    print(anova_table)
    print('\n')
    m_comp = pairwise_tukeyhsd(endog=df_temp[col2_replaced], groups=df_temp[col1_replaced], alpha=0.05)
    print(m_comp)

ModuleNotFoundError: No module named 'statsmodels'

In [ ]:
from scipy.stats import chi2_contingency, chi2
def plot_crosstab_and_show_dependency(df, col1, col2, no_plot=False):
    crosstab = pd.crosstab(df[col1], df[col2])
    col1 = str(col1)
    col2 = str(col2)
    if not no_plot:
        crosstab.plot(kind='bar', stacked=True, figsize=(5, 3))
        plt.title(f'{col1} vs {col2}')
        plt.show()

    stat, p, dof, expected =chi2_contingency(crosstab)
    print(f'p-value: {p}')
    print(f'dof: {dof}')

    prob = 0.975
    critical = chi2.ppf(prob, dof)
    print(f'probability: {prob}, critical: {critical}, stat: {stat}')

    alpha = 1 - prob
    print(f'alpha (significant): {alpha}, p-value: {p}')
    if p <= alpha:
        print(f'{col1} dependent {col2} (reject H0)')
    else:
        print(f'{col1} independent {col2}(fail to reject H0)')

In [ ]:
def num_univariate_analysis(data, col: str, plot_only=False, no_outlier_analysis=False):
    #Descriptive statistics
    col_median = data[col].median()
    col_mode = data[col].mode().iloc[0]
    col_range = data[col].max() - data[col].min()
    col_variance = data[col].var()
    col_skew = data[col].skew()
    col_kurt = data[col].kurt()

    if not plot_only:
        print(data[col].describe(include='all'))
        print(f"Median {col_median}")
        print(f"Mode {col_mode}")
        print(f"Range {col_range}")
        print(f"Var {col_variance}")
        print("****************************************************")

    # Histogram
    plt.figure(figsize=(8, 3))
    plt.subplot(1, 2, 1)
    sns.histplot(data[col], kde=True)
    # plt.subplot(1, 3, 2)
    # plt.hist(data[col])
    if data[col].nunique() < 10:
        plt.subplot(1, 2, 2)   
        plt.pie(data[col].value_counts(), labels=data[col].value_counts().index, autopct='%1.1f%%')
    plt.show()
    print(f"Skew {col_skew}")
    print(f"Kurt {col_kurt}")
    if col_skew > 0:
        print("Phân phối lệch phải")
    else:
        print("Phân phối lệch trái")
    
    if col_kurt > 0:
        print("Phân phối nhọn hơn phân phối chuẩn")
    else:
        print("Phân phối có độ nhọn thấp hơn phân phối chuẩn")
    print("****************************************************")

    if no_outlier_analysis:
        return
    col_Q1 = data[col].quantile(0.25)
    col_Q3 = data[col].quantile(0.75)
    col_IQR = col_Q3 - col_Q1
    
    if not plot_only:
        print(f"Q1 - Q3: {col_Q1} - {col_Q3}")
        print(f"IQR {col_IQR}")

    # Boxplot, Outliers
    plt.figure(figsize=(2,2))
    plt.boxplot(data[col])
    plt.show()
    col_outliers_upper_num = data[data[col] > (col_Q3 + 1.5*col_IQR)].shape[0]
    col_outliers_lower_num = data[data[col] < (col_Q1 - 1.5*col_IQR)].shape[0]
    col_outliers_per = (col_outliers_upper_num + col_outliers_lower_num)/data.shape[0] * 100
    print(f"Number of outliers: {col_outliers_upper_num + col_outliers_lower_num}")
    print(f"Number of upper outliers: {col_outliers_upper_num}")
    print(f"Number of lower outliers: {col_outliers_lower_num}")
    print(f"Percentage of outliers: {col_outliers_per}%")
    # vì outlier có tỉ lệ nhỏ (<1%) nên có thể không cần loại bỏ outlier

In [ ]:
class PlotType(enumerate):
    HIST = 'hist'
    PIE = 'pie'
    BAR = 'bar'

def cat_univariate_analysis(data:pd.DataFrame, fig_size=(10, 5), exclude_cols=[], num_cols=3, plot_type=PlotType.HIST):
    i = 0
    category_cols = [col for col in data.columns if data[col].dtype == 'object']
    num_rows = len(category_cols) // num_cols + 1
    fig = plt.figure(figsize=fig_size)
    for col in category_cols:
        if col in exclude_cols:
            continue
        i += 1
        fig.add_subplot(num_rows, num_cols, i)
        if plot_type == PlotType.HIST:
            sns.histplot(data[col])
        elif plot_type == PlotType.PIE:
            plt.pie(data[col].value_counts(), labels=data[col].value_counts().index, autopct='%1.1f%%')
        elif plot_type == PlotType.BAR:
            sns.barplot(x=data[col].value_counts().index, y=data[col].value_counts())
        plt.xticks(rotation=90)
        plt.title(col)
    plt.tight_layout()
    plt.show()

In [ ]:
# numbers_cols = [col for col in data.columns if data[col].dtype != 'object']
# description = data.describe()
# modes = data.mode().iloc[0] # (1,33)
# description.loc['mode'] = modes
# description.loc['var'] = data[numbers_cols].var()
# standard_error = data[numbers_cols].sem()
# description.loc['std'] = standard_error
# kurtosis = data[numbers_cols].kurtosis()
# description.loc['kur'] = kurtosis
# skewness = data[numbers_cols].skew()
# description.loc['skew'] = skewness
# range = data[numbers_cols].max() - data[numbers_cols].min()
# description.loc['range'] = range
# description

#### Descriptive Analysis

In [ ]:
# num_univariate_analysis(fact_order, ColumnNames.SalePrice)

fact_view = get_fact_view(fact_order, [ColumnNames.DateKey, ColumnNames.CustomerKey, ColumnNames.CityKey, ColumnNames.StateKey, ColumnNames.SaleAgentKey, ColumnNames.ProductCatalogKey, ColumnNames.ReturnReasonKey, ColumnNames.OrderStatusKey])
fact_view[ColumnNames.ProfitPerOrder] = fact_view[ColumnNames.ProfitPerUnit] * fact_view[ColumnNames.QuantityOrdered]
fact_view.rename(columns={'Order ID': ColumnNames.OrderID}, inplace=True)
fact_view.info()

NameError: name 'get_fact_view' is not defined

In [ ]:
class ReturnReason(enumerate):
    DontHave = 'Dont have'
    Defective = 'Defective'
    LateDelivery = 'Late Delivery'
    QualityIssue = 'Quality Issue'
    ReceivedWrongItem = 'Received Wrong Item'
    Unsatisfied = 'Unsatisfied'



#calculate profit loss by multiple reasons
# defective_profit_loss = fact_view[fact_view[ColumnNames.ReturnReasonName] == ReturnReason.Defective][ColumnNames.ProfitPerOrder].sum()
# print(f"Profit loss by defective: {defective_profit_loss}")
print(fact_view.columns)
fact_view_return = fact_view[fact_view[ColumnNames.OrderStatusName] == 'Cancelled']
profit_loss = fact_view_return.groupby([ColumnNames.ReturnReasonName]).agg(
    total_profit_loss_order = (ColumnNames.ProfitPerOrder, 'sum'),
    case_num = (ColumnNames.OrderID, 'count'),
    mean_profit_loss_order = (ColumnNames.ProfitPerOrder, 'mean'),
    mean_profit_loss_unit = (ColumnNames.ProfitPerUnit, 'mean'),
)
# remove the return reason DontHave
# profit_loss = profit_loss.drop(ReturnReason.DontHave) # if we not filter fact_view by OrderStatusName = 'Cancelled'
profit_loss.sort_values(by='total_profit_loss_order', ascending=False)

Index(['OrderID', 'Quantity Ordered', 'Manufacturer Price', 'Sale Price',
       'Total Profit (GMROI)', 'Date', 'Month', 'Day', 'WeekDayName',
       'Customer Last Name', 'Customer First Name', 'Phone', 'CityName',
       'State', 'SaleAgentName', 'ProductCatalogName', 'ReturnReasonName',
       'OrderStatusName', 'Profit Per Order'],
      dtype='object')


,total_profit_loss_order,case_num,mean_profit_loss_order,mean_profit_loss_unit
ReturnReasonName,,,,
Received Wrong Item,120758,73,1654.219178,156.027397
Late Delivery,114516,67,1709.194030,144.432836
Quality Issue,86517,44,1966.295455,165.295455
Defective,84690,59,1435.423729,133.050847
Unsatisfied,80963,66,1226.712121,127.909091


tỉ lệ hàng bị return về chiểm bao nhiêu % so với tổng case, tỉ lệ bao nhiêu % so với tổng case không bị return, tỉ lệ này có đang tăng

In [ ]:
product_catalog_names = fact_view[ColumnNames.ProductCatalogName].unique()
# create a dataframe of mean, min, max, quantiles, median, mode, of different ProductCatalogName
product_catalog_summary = fact_view.groupby(ColumnNames.ProductCatalogName).agg(
    count_return_by_defect = (ColumnNames.ReturnReasonName, lambda x: (x == ReturnReason.Defective).sum()),
    percent_return_by_defect = (ColumnNames.ReturnReasonName, lambda x: (x == ReturnReason.Defective).sum()/x.count()*100),
)
product_catalog_summary

,count_return_by_defect,percent_return_by_defect
ProductCatalogName,,
Books,6,1.481481
Clothing,6,1.315789
Electronics,8,1.687764
Home & Kitchen,10,2.380952
Personal Care,12,2.666667
Pets,10,2.469136
Sports & Outdoors,7,1.794872


In [ ]:
product_catalog_names = fact_view[ColumnNames.ProductCatalogName].unique()
reasons = ['Dont have', 'Defective', 'Late Delivery', 'Quality Issue', 'Received Wrong Item', 'Unsatisfied']
for reason in reasons:
    if reason == ReturnReason.DontHave:
        continue
    print(f'Return by reason: {reason}')
    print('****************************************************')
    fact_view_returned_by_reason = fact_view[fact_view[ColumnNames.ReturnReasonName] == reason]
    product_catalog_summary = fact_view_returned_by_reason.groupby(ColumnNames.ProductCatalogName).agg(
        total_profit_loss_order = (ColumnNames.ProfitPerOrder, 'sum'),
        case_num = (ColumnNames.OrderID, 'count'),
        mean_profit_loss_order = (ColumnNames.ProfitPerOrder, 'mean'),
        mean_profit_loss_unit = (ColumnNames.ProfitPerUnit, 'mean'),
    )

    # calculate the percentage of return by defective
    product_catalog_summary['percent_return_by_reason'] = product_catalog_summary['case_num']/fact_view_returned_by_reason.shape[0] * 100
    product_catalog_summary['percent_profit_order_loss'] = product_catalog_summary['total_profit_loss_order']/product_catalog_summary['total_profit_loss_order'].sum() * 100

    print(product_catalog_summary[['total_profit_loss_order', 'case_num', 'percent_profit_order_loss', 'percent_return_by_reason']].sort_values(by='total_profit_loss_order', ascending=False))

Return by reason: Defective
****************************************************
                    total_profit_loss_order  case_num  \
ProductCatalogName                                      
Personal Care                         30520        12   
Clothing                              12046         6   
Home & Kitchen                        11447        10   
Books                                 10539         6   
Pets                                   9334        10   
Electronics                            6222         8   
Sports & Outdoors                      4582         7   

                    percent_profit_order_loss  percent_return_by_reason  
ProductCatalogName                                                       
Personal Care                       36.037313                 20.338983  
Clothing                            14.223639                 10.169492  
Home & Kitchen                      13.516354                 16.949153  
Books                              

In [ ]:
print(fact_view.columns)
fact_view_return = fact_view[fact_view[ColumnNames.OrderStatusName] == 'Cancelled']
profit_loss = fact_view_return.groupby([ColumnNames.ProductCatalogName]).agg(
    total_profit_loss_order = (ColumnNames.ProfitPerOrder, 'sum'),
    case_num = (ColumnNames.OrderID, 'count'),
    mean_profit_loss_order = (ColumnNames.ProfitPerOrder, 'mean'),
    mean_profit_loss_unit = (ColumnNames.ProfitPerUnit, 'mean'),
    percent_profit_order_loss = (ColumnNames.ProfitPerOrder, lambda x: x.sum()/fact_view_return[ColumnNames.ProfitPerOrder].sum()*100),
    percent_case_num = (ColumnNames.OrderID, lambda x: x.count()/fact_view_return[ColumnNames.OrderID].count()*100),
)
# remove the return reason DontHave
# profit_loss = profit_loss.drop(ReturnReason.DontHave) # if we not filter fact_view by OrderStatusName = 'Cancelled'
profit_loss.sort_values(by='total_profit_loss_order', ascending=False)

Index(['OrderID', 'Quantity Ordered', 'Manufacturer Price', 'Sale Price',
       'Total Profit (GMROI)', 'Date', 'Month', 'Day', 'WeekDayName',
       'Customer Last Name', 'Customer First Name', 'Phone', 'CityName',
       'State', 'SaleAgentName', 'ProductCatalogName', 'ReturnReasonName',
       'OrderStatusName', 'Profit Per Order'],
      dtype='object')


,total_profit_loss_order,case_num,mean_profit_loss_order,mean_profit_loss_unit,percent_profit_order_loss,percent_case_num
ProductCatalogName,,,,,,
Personal Care,121144,56,2163.285714,176.125000,24.852906,18.122977
Electronics,78235,48,1629.895833,137.187500,16.050049,15.533981
Home & Kitchen,70390,49,1436.530612,130.448980,14.440633,15.857605
Books,62522,39,1603.128205,155.923077,12.826499,12.621359
Clothing,60078,42,1430.428571,133.142857,12.325108,13.592233
Pets,51219,37,1384.297297,143.162162,10.507669,11.974110
Sports & Outdoors,43856,38,1154.105263,126.894737,8.997136,12.297735
